# 1. Count-Based Word Representation

## 01. Bag of Words(BoW)
: **단어의 순서를 고려하지 않고** **출현 빈도**에만 집중해 단어를 표현(word representation)하는 방법

### 01-1. DTM/TDM 
: 문서를 구성하는 단어들이 몇번 나오는지 표현하는 행렬로 Vector화

In [5]:
text = ["나는 야구를 좋아합니다. 나는 축구를 좋아하지 않습니다.", 
        "친구는 야구와 축구를 좋아합니다. 야구보다 축구를 더 좋아합니다.", 
        "나는 축구를 더 좋아합니다. 영국 축구 리그보다 이탈리아 축구 리그를 더 좋아합니다.", 
        "나는 농구를 좋아하지 않습니다. 배구도 좋아하지 않습니다.", 
        "나는 액션영화를 좋아합니다.", 
        "어제 비빕밥을 먹었습니다."]

stop_words = list('은는이가을를도.,')

In [7]:
# Okt 모델 활용
# 1) import
from konlpy.tag import Okt
# 2) Okt 모델을 불러와서 okt변수로 지정
okt = Okt()  # 함수로 불러오기
# 3) 토크나이저(tokenizer) 함수 정의하기
# tokenizer라는 함수를 만들고 doc를 매개변수(parameter)로 지정
def tokenizer (doc):
    return okt.morphs(doc)  # 형태소 단위로 토큰화 - 메소드 지정 


tokenizer(text[0])

['나', '는', '야구', '를', '좋아합니다', '.', '나', '는', '축구', '를', '좋아하지', '않습니다', '.']

In [ ]:
# 전처리기 객체 생성
# 1) import
from sklearn.feature_extraction.text import CountVectorizer   # Stopword를 한번에 처리해주는 기능이 있음.
import pandas as pd
# 2) CountVectorizer 전처리기 생성
cv = CountVectorizer(
    tokenizer=_________, # 앞에서 만들어둔 함수 불러오기 # 문서를 받아서 토큰화 처리하는 callable 전달.
    stop_words = __________,  # 불용어는 앞에 stop_words 변수에 넣음.
    token_pattern=None
)
# 3) 전처리기 학습 
# 앞에 만들어둔 text 넣기
cv._____(____)   # 학습할 때 쓰는 메소드(fit? transform? 뭘 써야 할까여)

In [ ]:
# 4-1) 어휘사전
cv._________   # 단어(vocabulary)와 그에 따른 index를 dictionary 형태로 반봔

# 4-2) 어휘사전 정렬
# vocabulary_.items()는 (단어, 인덱스) 형태의 튜플들로 구성된 iterable 객체를 반환
# key함수: 자료구조 원소를 받아서 정렬할 때 사용할 값을 반환
# key값은 lambda 활용해서 튜플 형태 중 인덱스 값 호출 (값, 인덱스)
dict(sorted(cv.vocabulary_.items(), key=______ x: x[_])) 

In [ ]:
# 5) 변환(transform)+ array 형태로 바꾸기
cv._______(text)_______()

# 5-2) feature 이름(토큰화된 단어) 조회하기
cv._______________() 

#5-3) dataframe으로 만들어보기
DTM_df = pd.DataFrame(
    cv.transform(text).toarray(),
    columns= cv.get_feature_names_out(),
    index= [f"문서{i+1}" for i in range(len(text))]
)  
DTM_df


> #### n-gram
>
> -   N 개의 단어(token)을 묶어서 하나의 토큰으로 처리하는 방식을 n-gram이라고 한다. (n은 몇개 토큰을 하나의 단위로 묶을지 개수)
>     -   uni-gram (n=1), bi-gram (n=2), tri-gram (n=3), 4개부터는 n-gram으로 표기(4-gram, 5-gram, ..)

In [ ]:
#######################
##n-gram 적용
#######################
cv = CountVectorizer(
    tokenizer=tokenizer,
    stop_words=stop_words,
    token_pattern=None,
    ngram_range=(1,3)  #ngram: 1, 2, 3
)

In [ ]:
# 학습
cv.fit(text) # 어휘사전 생성

In [ ]:
cv.get_feature_names_out()   

# 여기서 질문! n_gram을 적용하지 않을 때와 토큰에 어떤 차이가 있나요?

In [ ]:
# dataframe으로 만들어보기
ngram_df = pd.DataFrame(
    cv.transform(text).toarray(),
    columns=cv.get_feature_names_out()
)
ngram_df

### 01-2. TF-IDF

: 개별 문서에 많이 나오는 단어가 높은 값을 가지도록 하되 동시에 여러 문서에 자주 나오는 단어에는 페널티를 주는 방식

> 문서에서 나온 횟수가 같더라도, 다른 문서에서 더 많이 나왔을 때, 그 가중치를 곱해줌

-   TF (Term Frequency) 정의: 해당 단어가 **해당 문서에** 몇번 나오는지를 나타내는 지표
-   DF (Document Frequency) 정의: 해당 단어가 **몇개의 문서에** 나오는지를 나타내는 지표
-   IDF (Inverse Document Frequency) 정의: DF에 역수로 $\cfrac{\text{전체 문서수}}{\text{해당 단어가 나오는 문서수}}$
-   TF-IDF 정의: $TF * \left(\log \cfrac{\text{전체 문서수}}{\text{해당 단어가 나오는 문서수}} \right)$

   -   log는 전체 문서의 수가 많으면 값의 단위가 너무 커지므로 log를 취한다.
    -   scikit-learn의 경우 분모가 0이 되는 것을 방지하기 위해 **분모에 1을 더하고** $\log(0)$도 계산이 안되므로 **분자에도 1을 더했으며** 그 계산 결과에 **1을 더하여 계산**함.
        -   $TF * \left(\log \cfrac{\text{전체 문서수 + 1}}{\text{해당 단어가 나오는 문서수 + 1}} + 1\right)$

In [ ]:
# 1. TfidfVectorizer import
from sklearn.feature_extraction.text import TfidfVectorizer
#2. TfidfVectorizer 전처리기 생성
tfidf = TfidfVectorizer(
    tokenizer=tokenizer,
    stop_words=stop_words,
    token_pattern=None
)

# 3. fit & transform
#입력: [문서1, 문서2, ...]
tfidf.fit(text)   # 어휘사전 생성
result = tfidf.transform(text) # 변환

In [ ]:
# 4. dataframe으로 만들기
tfidf_df = pd.DataFrame(
    result.toarray(),
    columns= tfidf.get_feature_names_out(),
    index= [f"문서{i+1}" for i in range(len(text))]
)
tfidf_df.iloc[:2]   
# 여러 문서에 나올수록 값이 더 작아짐. 
# 즉 횟수가 더 많은 값이 나올수록 영향을 적게 미치게됨. 
# ex. 좋아합니다: 6번, 0.287558 / 축구: 5번, 0.33557

# 2. Word Embedding
: 앞선 Count-based는 출현 빈도만 나타내기 때문에 문장을 모두 파악하는데 정보가 부족하다.<br>
빈도수 이외에 의미의 유사성이 반영될 수 있도록 비슷한 의미를 가지는 단어는 비슷한 값들로 구성되도록 하기 위해 워드 임베딩 사용

**단어를 밀집 벡터(dense vector)의 형태로 표현하는 방법을 워드 임베딩(word embedding)** 이라고 한다. 워드 임베딩 결과로 나온 밀집 벡터를 **임베딩 벡터(embedding vector)** 라고 한다.

## 02. Word2Vec
: 딥러닝 기반 word embedding 방식. 딥러닝 모델은 `입력층-은닉층-출력층` 의 단순한 구조

### 02-1. CBOW(Continous Bag of Words)
:  **주변 단어들**로부터 중심 단어를 예측하는 방식

#### Dataset 구성
- CBOW/Skip-gram 모델 학습 데이터셋은 모두 token들이 one-hot encoding 되 있어야 한다.  

#### Window size 설정
- Window size를 설정 한 뒤 문장을 중심 단어를 뒤로 이동 시키면서 중심단어와 주변단어들을 추출해 데이터셋을 만든다.
- 지정한 개수(window) 만큼씩 이동하면서 어떤 작업을 진행하는 것을 **Sliding window 방식** 이라고 한다.

<img src="figures/word2vec_cbow.png" width="800">

### 02-2. Skip-gram

중심단어를 이용해 그 주변단어를 예측하는 모델을 구성한다.

<img src='figures/word2vec_skipgram.png' width="800">

- 모델의 입력으로 중심단어의 one-hot vector가 들어가고 모델을 주변단어를 추론한다. window size 가 2라면 총 4개의 주변단어를 추론한다.
- 학습 할 때는 각 주변단어들에 대한 개별 loss를 계산하고 그 합계를 최종 loss하여 $W_{in}$과 $W_{out}$ 을 update 한다.

| 항목                | **CBOW (Continuous Bag of Words)** | **Skip-Gram**                             |
| ----------------- | ---------------------------------- | ----------------------------------------- |
| 🔁 **예측 방향**      | 주변 단어 → 중심 단어                      | 중심 단어 → 주변 단어                             |
| 🧾 **입력(Input)**  | 주변 단어들 (Context Words)             | 중심 단어 (Target Word)                       |
| 🎯 **출력(Output)** | 중심 단어 (Target Word)                | 주변 단어들 (Context Words)                    |
| ⚙️ **학습 방식**      | 주변 단어 벡터들의 평균을 통해 예측               | 하나의 중심 단어로 여러 주변 단어 예측                    |
| ⏱ **학습 속도**       | 상대적으로 빠름                           | 상대적으로 느림                                  |
| 🔍 **희귀 단어 표현**   | 상대적으로 부정확                          | 희귀 단어 표현에 강함                              |
| 📦 **말뭉치 크기 적합성** | 작은 말뭉치에 유리                         | 큰 말뭉치에 유리                                 |
| 🎯 **적합한 사용 목적**  | 일반적인 단어 표현 학습                      | 정교한 단어 표현 학습                              |
| 💬 **문맥 정보 처리**   | 주변 단어 평균 처리 (정보 손실 가능성 있음)         | 각 주변 단어를 독립적으로 처리                         |
| 🛠 **주요 활용 기법**   | Hierarchical Softmax 등             | Negative Sampling, Hierarchical Softmax 등 |


- 입력 단어는 **one-hot vector** 다. 그래서 단어 index만 1이고 나머진 모두 0으로 구성된다.
이  입력 one-hot vector와 가중치 행렬 $W_{in}$이 가중합(행렬곱)을 계산하면 가중치 행렬에서 그 단어 index의 행(one-hot vector의 1의 index의 행)  행을 가져오는 것이 된다. 그래서 word2vec의 hidden layer를 계산하는 작업($X \cdot W_{in}$)은 **가중치 행렬 $W_{in}$에서 해당 단어에 해당하는 행을 찾는(lookup) 작업**을 하는 것이 된다.<br>
> (1,4)@     (4@3)-> 가중치값인데 결국, 1에 해당되는 가중치값이  임베딩 vector가 된다. <br>
> 위의 예시에서는 (1,10000)@(10000,100)이고,그 임베딩 vector의 값은 0번 : 가중치에서 가장 첫번째 줄
    - ex)
\begin{align} 
\left[
\begin{matrix}
    0 & 0& 1 & 0
\end{matrix}
\right] \cdot \left[
\begin{matrix}
0.1 & 0.1 & 0.1 \\
0.2 & 0.2 & 0.2 \\
0.3 & 0.3 & 0.3 \\
0.4 & 0.4 & 0.4
\end{matrix}
\right] = \left[
\begin{matrix}
0.3 & 0.3 & 0.3
\end{matrix}
\right] \\
\text{행: 단어(4), 열: embedding 차원(3)}
\end{align}    
- Word2Vec의 학습은 **가중치 행렬의 각 행들이 단어들의 word embedding vector**가 되도록 **주변단어와 중심단어의 관계로 학습**하는 것이다.
- 학습이 완료 되면 $W_{in}$ 이나 $W_{out}$ 파라미터를 word embeding vector로 사용한다.

## ✅ 임베딩 백터화 정리

다음은 one-hot 벡터와 가중치 행렬 $W_{in}$ 간의 행렬곱으로 임베딩 벡터를 얻는 과정

### 📌 입력 벡터 (one-hot)

$$
x = \begin{bmatrix}
0 & 0 & 1 & 0
\end{bmatrix}
$$

### 📌 가중치 행렬 $W_{in}$

$$
W_{in} =
\begin{bmatrix}
0.1 & 0.1 & 0.1 \\
0.2 & 0.2 & 0.2 \\
0.3 & 0.3 & 0.3 \\
0.4 & 0.4 & 0.4
\end{bmatrix}
$$

### 📌 행렬곱 결과 (임베딩 벡터 $h$)

$$
h = x \cdot W_{in}
=
\begin{bmatrix}
0.3 & 0.3 & 0.3
\end{bmatrix}
$$

즉, 입력된 단어가 **"cherry"** (인덱스 2)일 경우,
가중치 행렬 $W_{in}$의 2번째 행을 그대로 가져와서
**임베딩 벡터로 사용하는 것과 동일합니다.**
